In [2]:
import numpy as np
import trimesh
import matplotlib.pyplot as plt
import taichi as ti
from trimesh.transformations import rotation_matrix
from mesh_to_sdf import mesh_to_voxels
from scipy.interpolate import RegularGridInterpolator
from time import time_ns
from numpy.linalg import norm
from fluid_grid import FluidGrid
from pathlib import Path
ti.init(arch=ti.cpu, debug=True) # cpu, to avoid insane copying


[Taichi] version 1.6.0, llvm 15.0.1, commit f1c6fbbd, win, python 3.10.9
[Taichi] Starting on arch=x64


#### Sampling the boundary of a triangle mesh with the method of (Bell, 2005)
See section 3.3.1 in that paper
Basic idea:
* get an SDF (signed distance function) approximation of the mesh
* put particles randomly on the mesh surface according to the area of each triangle
* adjust particle positions on the mesh to obtain a nice & even spread of particles
* this is with a simulation approach where velocities are calulated per particles


In [3]:
#Note: the "Box.glb" file is from \
#https://github.com/KhronosGroup/glTF-Sample-Models/blob/master/2.0/Box/glTF-Binary/Box.glb
# Due to problems with the sdf library, I rotated the Box first.
scene = trimesh.load("boundary/Box.glb")
box = scene.dump()[0]
rotm = rotation_matrix(np.pi / 4, (0, 1, 0), box.centroid)
box.apply_transform(rotm)
box.export("boundary/Box_rot.glb")

b'glTF\x02\x00\x00\x00T\x05\x00\x00\x88\x03\x00\x00JSON{"scene":0,"scenes":[{"nodes":[0]}],"asset":{"version":"2.0","generator":"https://github.com/mikedh/trimesh"},"accessors":[{"componentType":5125,"type":"SCALAR","bufferView":0,"count":36,"max":[23],"min":[0]},{"componentType":5126,"type":"VEC3","byteOffset":0,"bufferView":1,"count":24,"max":[0.7071067690849304,0.5,0.7071067690849304],"min":[-0.7071067690849304,-0.5,-0.7071067690849304]}],"meshes":[{"name":"Mesh","extras":{"units":"meters","from_gltf_primitive":false,"name":"Mesh","node":"1"},"primitives":[{"attributes":{"POSITION":1},"indices":0,"mode":4}]}],"materials":[{"pbrMetallicRoughness":{"baseColorFactor":[0.8,0.0,0.0,1.0],"metallicFactor":0.0},"name":"Red","doubleSided":false}],"nodes":[{"name":"world","children":[1]},{"name":"Mesh","mesh":0}],"buffers":[{"byteLength":432}],"bufferViews":[{"buffer":0,"byteOffset":0,"byteLength":144},{"buffer":0,"byteOffset":144,"byteLength":288}]}  \xb0\x01\x00\x00BIN\x00\x00\x00\x00\x00\x

In [4]:
# calculate SDF grid
mesh_path = Path("boundary/Box_rot.glb")
precomp_path = mesh_path.with_name("precomp_" + mesh_path.name).with_suffix(".npy")

mesh = trimesh.load(mesh_path)
if not precomp_path.exists():
    voxels, gradients = mesh_to_voxels(mesh, 32, pad=False, return_gradients=True)
    with open(precomp_path, "wb") as wf:
        np.save(wf, voxels)
        np.save(wf, gradients)
# load
with open(precomp_path, "rb") as rf:
    voxels = np.load(rf)
    gradients = np.load(rf)

In [5]:
if type(mesh) is trimesh.Scene: 
    mesh = mesh.dump()[0]
mesh.apply_translation(- mesh.bounding_box.centroid)
verts = mesh.vertices 
faces = mesh.faces

In [6]:
# set up SDF interpolation
# Note: the SDF calculation algo is different from the one described in the paper
# probably does not matter. what does is that the mesh is scaled into a -1.0 to 1.0 cube
# hence the output grid needs be inversely scaled 
voxel_scale =  np.max(mesh.bounding_box.extents) / 2
x = np.linspace(-1.0, 1.0, 32) * voxel_scale
y = np.linspace(-1.0, 1.0, 32) * voxel_scale
z = np.linspace(-1.0, 1.0, 32) * voxel_scale
inter_voxels = RegularGridInterpolator((x, y, z), voxels, bounds_error=False, fill_value=None )
inter_grad   = RegularGridInterpolator((x, y, z), gradients, bounds_error=False, fill_value=None )

#### Simulation part

For each particle, the velocity is given by v_r + v_f
v_r is the inter-particle force. \
v_r = neigh_sum[ kernel(p_i - p_j) * (p_i - p_j).normalized() ] \
v_f keeps particles to the mesh surface using the SDF data. \
v_f = -sdf(p_i) * sdf_normal(p_i)

In [7]:
# next, set up a simulation a required

particle_radius = 0.04
max_particles_per_face = 1024

@ti.func
def cubic_kernel(r_norm, h):
    # implementation details borrowed from SPH_Taichi
    # use ps.smoothing_radius to calculate the kernel weight of particles
    # for now, sum over nearby particles
    w = ti.cast(0.0, ti.f32)
    k = 8 / np.pi
    k /= ti.pow(h, 3)
    q = r_norm / h
    if q <= 1.0:
        if q <= 0.5:
            q2 = ti.pow(q, 2)
            q3 = ti.pow(q, 3)
            w = k * (6.0 * q3 - 6.0 * q2 + 1)
        else:
            w = k * 2 * ti.pow(1 - q, 3.0)
    return w

# a class to hold all the sim data
@ti.data_oriented
class Data:
    def __init__(self, mesh_vertices, mesh_faces) -> None:
        numFaces = mesh_faces.shape[0]
        self.time_step = 0.015
        self.h = particle_radius * 4.0
        self.origin = ti.Vector([-3.0]*3)
        self.grid_end = ti.Vector([3.0]*3)
        self.grid_spacing = self.h * 1.2
        self.fg = FluidGrid(self.origin, self.grid_end, self.grid_spacing)
        print("grid size:", self.fg.num_cells)
        print("num faces:", numFaces)
        self.particle_nums = np.empty(dtype=int, shape=numFaces)
        tmp_pos = np.empty(shape=numFaces, dtype=object)
        particle_sum = 0
        # here we init particles per face
        for tri_idx in range(numFaces):
            print("\r", tri_idx, end="")
            face = mesh_faces[tri_idx]
            x0 = mesh_vertices[face[0]]
            x1 = mesh_vertices[face[1]]
            x2 = mesh_vertices[face[2]]
            e1 = x1 - x0
            e2 = x2 - x0

            # calculate D*A / (pi*r^2)
            sample_density = 1.3
            area = norm(np.cross(e1, e2)) / 2.0
            numParticles = int(sample_density * area / (np.pi * particle_radius**2))
            numParticles = min(numParticles, max_particles_per_face)
            
            pos = np.empty(shape=(numParticles, 3))

            rand = np.random.uniform(size= 2 * numParticles).reshape((numParticles, 2))
            tri_normal = np.cross(e1, e2)
            for i in range(numParticles):
                scale = np.random.random()*0.2
                x = e1 * rand[i][0] + (1-rand[i][0])*rand[i][1]*e2
                pos[i] = ti.Vector(x+x0+tri_normal*scale)
            self.particle_nums[tri_idx] = numParticles
            particle_sum += numParticles
            tmp_pos[tri_idx] = pos
        print()
        print("num particles", particle_sum)
        self.pos = ti.Vector.field(n=3, dtype=float, shape=particle_sum)
        self.vel = ti.Vector.field(n=3, dtype=float, shape=particle_sum)
        # write all the positions into a single ti.field
        offset = 0
        for pos_ in tmp_pos:
            for i in range(pos_.shape[0]):
                self.pos[offset + i] = pos_[i]
            offset += pos_.shape[0]

    @ti.kernel
    def update_positions(self):
        for i in ti.grouped(self.vel):
            self.pos[i] += self.time_step * self.vel[i]
            self.pos[i] = ti.math.clamp(self.pos[i], self.origin, self.grid_end)
            
    @ti.func
    def aux_update_velocities(self, i, j, vr:ti.template()):
        if i[0] != j:
            r = (self.pos[i] - self.pos[j])
            vr += cubic_kernel(r.norm(), self.h) * r.normalized()

    @ti.kernel
    def update_vr(self): # V_r part
        for i in ti.grouped(self.vel):
            vr = ti.Vector([0.0]*3)
            self.fg.for_all_neighbors(i, self.pos, self.aux_update_velocities, vr, self.h)
            if vr.norm() > self.h * 2:
                vr = vr / vr.norm() * self.h * 2
            self.vel[i] += vr

    def update_velocity(self):
        for i in range(self.pos.shape[0]):
            x_np = self.pos[i].to_numpy()

            phi = inter_voxels(x_np)
            n = inter_grad(x_np)
            v_f = -phi * n
            self.vel[i] = v_f[0, :] * 7.0
        self.update_vr()

data = Data(verts, faces)

Creating a grid with dimension 33, 33, 33
grid size: 35937
num faces: 12
 11
num particles 1548


In [8]:
# the window (im sorry that you can only open the window once)

vertices = ti.Vector.field(n=3, dtype=float, shape=verts.shape[0])
vertices.from_numpy(verts)
indices = ti.field(int, shape=faces.shape[0] * 3)
indices.from_numpy(faces.flatten())

window = ti.ui.Window("display", res=(1600,1600), vsync=True)
camera = ti.ui.Camera()
camera.up(0.0, 1.0, 0.0)
camera.position(-3.0, 0.5, 0.5)
camera.lookat(0,0.5,0)
canvas = window.get_canvas()
scene = ti.ui.Scene()

particle_sim = False

iteration = 0
while window.running:
    if window.is_pressed(ti.ui.SPACE, ' '): particle_sim = True
    if window.is_pressed(ti.ui.ALT): particle_sim = False
    camera.track_user_inputs(window, movement_speed=0.03, hold_key=ti.ui.RMB)
    scene.set_camera(camera)
    scene.ambient_light((0.8, 0.8, 0.8))
    scene.mesh(vertices=vertices, indices=indices ,color=(1.0,0.0,0.0), show_wireframe=True)
    if particle_sim:
        data.update_velocity()
        data.update_positions()
        data.fg.update_grid(data.pos)
        
    scene.particles(data.pos, radius=particle_radius*1.0)
    canvas.scene(scene)

    window.show()
    #particle_sim = False
    # decreasing timestep (experiment)
    data.time_step = 0.08 / (iteration + 1) ** 0.2
    iteration += 1


: 